In [50]:
!pip freeze |  grep scikit-learn

In [2]:
!python -V

Python 3.11.13


In [18]:
import pickle
import pandas as pd
import numpy as np

In [22]:
with open('lin_reg.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [36]:
categorical = ['PULocationID','DOLocationID']

def read_data(filename, year, month):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype(str)
    return df
    

In [41]:
year = 2023
month = 3
filename = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02}.parquet' 
print(filename)
df = read_data(filename,year,month)
# df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
print(df[:5])


https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-03-01 00:06:43   2023-03-01 00:16:43              1.0   
1         2  2023-03-01 00:08:25   2023-03-01 00:39:30              2.0   
2         1  2023-03-01 00:15:04   2023-03-01 00:29:26              0.0   
3         1  2023-03-01 00:49:37   2023-03-01 01:01:05              1.0   
4         2  2023-03-01 00:08:04   2023-03-01 00:11:06              1.0   

   trip_distance  RatecodeID store_and_fwd_flag PULocationID DOLocationID  \
0           0.00         1.0                  N          238           42   
1          12.40         1.0                  N          138          231   
2           3.30         1.0                  N          140          186   
3           2.90         1.0                  N          140           43   
4           1.23         1.0                  N           79          137   

   pay

In [27]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [20]:
std_dev = np.std(y_pred)
print(f"Standard Deviation of predicted duration: {std_dev}")

Standard Deviation of predicted duration: 6.24751524449896


In [43]:
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'predicated_duration': y_pred

})

In [45]:
output_file = 'output_file.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
